The first step is always to find the bottlenecks in your code, via _profiling_: analyzing your code by measuring the execution time of its parts.


Tools:
------

2. `cProfile`
4. `snakeviz`
1. [`line_profiler`](https://github.com/rkern/line_profiler)
3. `timeit`



```console
pip install line_profiler
```

In [2]:
import numpy
from time import sleep

def sleepy(time2sleep):
    sleep(time2sleep)
    
def supersleepy(time2sleep):
    sleep(time2sleep)
    
def randmatmul(n=1000):
    a = numpy.random.random((n,n))
    b = a @ a
    return b
    
def useless(a):
    if not isinstance(a, int):
        return
    
    randmatmul(a)
    
    ans = 0
    for i in range(a):
        ans += i
        
    sleepy(1.0)
    supersleepy(2.0)
        
    return ans

## using `cProfile`

[`cProfile`](https://docs.python.org/3.4/library/profile.html#module-cProfile) is the built-in profiler in Python (available since Python 2.5).  It provides a function-by-function report of execution time. First import the module, then usage is simply a call to `cProfile.run()` with your code as argument. It will print out a list of all the functions that were called, with the number of calls and the time spent in each.

from the command line:
`python -m cProfile -s tottime script.py`

to output in the file
`python -m cProfile -o output.pstats script.py`

to create a graph of dependencies + timing:
`gprof2dot -f pstats output.pstats | dot -Tpng -o output.png`

In [3]:
import cProfile

cProfile.run('useless(3000)')

         11 function calls in 3.764 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.606    0.606    0.744    0.744 <ipython-input-2-3d4635a3d65c>:10(randmatmul)
        1    0.013    0.013    3.764    3.764 <ipython-input-2-3d4635a3d65c>:15(useless)
        1    0.000    0.000    1.002    1.002 <ipython-input-2-3d4635a3d65c>:4(sleepy)
        1    0.000    0.000    2.005    2.005 <ipython-input-2-3d4635a3d65c>:7(supersleepy)
        1    0.000    0.000    3.764    3.764 <string>:1(<module>)
        1    0.000    0.000    3.764    3.764 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        2    3.007    1.503    3.007    1.503 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.139    0.139    0.139    0.139 {method 'random_sample' of 'mtrand.RandomState' 

## using `snakeviz`

In [4]:
%load_ext snakeviz

In [5]:
%snakeviz useless(3000)

 
*** Profile stats marshalled to file '/var/folders/mc/0c22tdns1s38xgw4sr17cmx80000gn/T/tmp88yl02kq'. 
Embedding SnakeViz in the notebook...


## using `line_profiler`

`line_profiler` offers more granular information than `cProfile`: it will give timing information about each line of code in a profiled function.

### For a pop-up window with results in notebook:

IPython has an `%lprun` magic to profile specific functions within an executed statement. Usage:
`%lprun -f func_to_profile <statement>` (get more help by running `%lprun?` in IPython).

In [6]:
%load_ext line_profiler
%lprun -f sleepy -f supersleepy useless(1000)

### Write results to a text file

In [6]:
%lprun -T timings.txt -f sleepy useless(1000)


*** Profile printout saved to text file 'timings.txt'. 


## Profiling on the command line

Open file, add `@profile` decorator to any function you want to profile, then run

```console
kernprof -l script_to_profile.py
```

which will generate `script_to_profile.py.lprof` (pickled result).  To view the results, run

```console
python -m line_profiler script_to_profile.py.lprof
```

In [7]:
from IPython.display import IFrame
IFrame('http://localhost:8888/terminals/1', width=800, height=700)

## `timeit`

```python
python -m timeit "print(42)"
```


In [8]:
# line magic
%timeit x=10

12.2 ns ± 0.653 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [9]:
%%timeit 
# cell magic

x=10
a='hello'
d=[1,2,3]

64.7 ns ± 3.05 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [10]:
import requests
import cProfile
 
 
def facebook():
    requests.get('https://facebook.com')
 
 
def google():
    requests.get('https://google.com')
def twitter():
    requests.get('https://twitter.com')
def vk():
    requests.get('https://vk.com')
 
 
def main():
    facebook()
    google()
    twitter()
    vk()

cProfile.run('main()')

         4815 function calls (4583 primitive calls) in 60.017 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:103(release)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:143(__init__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:147(__enter__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:151(__exit__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:157(_get_module_lock)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:176(cb)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:211(_call_with_frames_removed)
       12    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:222(_verbose_message)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:307(

      163    0.000    0.000    0.000    0.000 {method 'encode' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {method 'endswith' of 'str' objects}
        4    0.000    0.000    0.000    0.000 {method 'extend' of 'list' objects}
        6    0.000    0.000    0.000    0.000 {method 'find' of 'bytearray' objects}
       12    0.000    0.000    0.000    0.000 {method 'find' of 'str' objects}
        2    0.000    0.000    0.000    0.000 {method 'format' of 'str' objects}
       33    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
        4    0.000    0.000    0.000    0.000 {method 'isalnum' of 'str' objects}
        4    0.000    0.000    0.000    0.000 {method 'items' of 'collections.OrderedDict' objects}
       14    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
       24    0.000    0.000    0.000    0.000 {method 'join' of 'str' objects}
        7    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
      11

ConnectionError: HTTPSConnectionPool(host='facebook.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10cf4ae10>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [13]:
requests.get('https://facebook.com')

<Response [200]>